<a href="https://colab.research.google.com/github/rajy4683/EVAP2/blob/master/EVA6Session3_FirstAt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!nvidia-smi

Fri May 21 18:58:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    34W / 250W |   1179MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
!pip install torchsummary
from torchsummary import summary

from PIL import Image
import os
import numpy as np
import torch
import codecs
import string
import gzip
import lzma
from typing import Any, Callable, Dict, IO, List, Optional, Tuple, Union
from urllib.error import URLError

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
### Returns one element from MNIST dataset along with label, and random number
class MNISTAdderDataSet():
    training_file = 'training.pt'
    test_file = 'test.pt'
    classes = ['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four',
               '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']
    def __init__(
            self,
            path_name,
            train,
            transform,
            target_transform=None,
    ):
        """
        Args:
            path_name: String - Path to the file containing the training.pt and test.pt,
            train: Boolean - Indicates whether to create training Dataset or Test Dataset,
            transform: Object/None - Any torch.transform object,
            target_transform: Object/None - Any torch.transform object (default: None),

        Returns:
            None
        """
        self.train = train
        self.path_name = path_name
        self.transform = transform
        self.target_transform = target_transform
        if self.train:
            data_file = self.training_file
        else:
            data_file = self.test_file

        self.data, self.targets = torch.load(os.path.join(self.path_name, data_file))
    
    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple:
                image: transformed image as numpy.array,
                test_ip: one-hot encoded array of length 10 with a random position set to 1,
                target: class label of the image 
                result_add: sum of the random number and class label
        """
        #random_int = torch.randint(0,9,(1,))
        random_int = np.random.randint(0,9, (1,))[0]
        test_ip = torch.zeros((10,))
        test_ip[random_int ]=1
        result_add = torch.zeros((19,))
        
        img, target = self.data[index], int(self.targets[index])

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img.numpy(), mode='L')

        if self.transform is not None:
            img = self.transform(img)

        # if self.target_transform is not None:
        #     target = self.target_transform(target)
        
        result_add = random_int+target

        return img, test_ip, target, result_add

    def __len__(self) -> int:
        return len(self.data)

In [5]:
class MNISTAdderNet(nn.Module):
    def __init__(self,start_channels=32, 
                 exponetate_layers=True, 
                 max_random_value = 10, 
                 adder_classes=19,
                 hidden_dim=20):
        super(MNISTAdderNet, self).__init__()
        self.start_channels = start_channels
        self.multiplier = 2
        self.hidden_dim = hidden_dim

        if (exponetate_layers == False):
            self.multiplier = 1
        
        self.conv1 = nn.Conv2d(1, self.start_channels, 3, padding=1) # input - 3*28*28 Output: 16*28*28
        self.conv1_bn = nn.BatchNorm2d(self.start_channels)
        self.conv2 = nn.Conv2d(self.start_channels, self.start_channels*self.multiplier, 3, padding=1) # input - 16*28*28 Output: 32*28*28
        self.conv2_bn = nn.BatchNorm2d(self.start_channels*self.multiplier)

        self.start_channels = self.start_channels * self.multiplier
        self.pool1 = nn.MaxPool2d(2, 2) # input - 32*28*28 Output: 32*14*14

        self.conv3 = nn.Conv2d(self.start_channels, self.start_channels*self.multiplier, 3, padding=1)# input - 32*14*14 Output: 64*14*14
        self.conv3_bn = nn.BatchNorm2d(self.start_channels*self.multiplier)
        self.start_channels = self.start_channels * self.multiplier
        self.conv4 = nn.Conv2d(self.start_channels, self.start_channels*self.multiplier, 3, padding=1)# input - 64*14*14 Output: 128*14*14
        self.conv4_bn = nn.BatchNorm2d(self.start_channels*self.multiplier)
        self.start_channels = self.start_channels * self.multiplier
        self.pool2 = nn.MaxPool2d(2, 2)# input - 128*14*14 Output: 128*7*7

        self.conv5 = nn.Conv2d(self.start_channels, self.start_channels*self.multiplier, 3)# input - 128*7*7 Output: 256*5*5
        self.conv5_bn = nn.BatchNorm2d(self.start_channels*self.multiplier)
        self.start_channels = self.start_channels * self.multiplier
        self.conv6 = nn.Conv2d(self.start_channels, self.start_channels*self.multiplier, 3)# input - 256*5*5 Output: 512*3*3
        self.conv6_bn = nn.BatchNorm2d(self.start_channels*self.multiplier)
        self.start_channels = self.start_channels * self.multiplier

        self.conv7 = nn.Conv2d(self.start_channels, 10, 3)# input - 512*3*3 Output: 10*1*1
        ### Fully connected layers for performing addition operation 
        self.embed_layer = nn.Linear(max_random_value, self.hidden_dim) # input - 10*1 Output: 20*1
        self.conv_embed_layer = nn.Linear(max_random_value, self.hidden_dim) # input - 10*1 Output: 20*1 
        self.final_adder_layer = nn.Linear(2*self.hidden_dim, adder_classes) # input - 20*1 Output: 19*1

    def forward(self, x, rand_inp):
        x = F.relu(self.conv1_bn(self.conv1(x))) ### Layer 1 ---->(Relu(BN(Conv)))
        x = self.pool1(F.relu(self.conv2_bn(self.conv2(x)))) ### Layer 2---> MaxPool((Relu(BN(Conv))))
        x = F.relu(self.conv3_bn(self.conv3(x))) ### Layer 3 ---->(Relu(BN(Conv)))
        x = self.pool2(F.relu(self.conv4_bn(self.conv4(x)))) ### Layer 4---> MaxPool((Relu(BN(Conv))))
        #########
        x = F.relu(self.conv5_bn(self.conv5(x))) ### Layer 5---> (Relu(BN(Conv)))
        x = F.relu(self.conv6_bn(self.conv6(x))) ### Layer 6 ---> (Relu(BN(Conv)))
        x = self.conv7(x) #### Final Layer (BN(Conv)))
        x = x.view(-1, 10)
        conv_to_fc = F.relu(self.conv_embed_layer(x)) ### We pass the predicted class through an FC to create an embedding 
        adder_output = F.relu(self.embed_layer(rand_inp)) ### Random Input is passed through an FC layer to create an embedding
        concat_layers = torch.cat((conv_to_fc, adder_output), dim=1)  ### Concat above two inputs for passing through final linear layer
        final_adder_output = self.final_adder_layer(concat_layers) ### Outputs a class between 0 and 18. Max sum of digits =18 and min = 0

        return F.log_softmax(x), F.log_softmax(final_adder_output)

## Dataloaders

In [8]:
batch_size=128

### For training and test data we normalize the images by the MNIST dataset's mean and variance.
my_ds = MNISTAdderDataSet('/content/data/MNIST/processed', True, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,)) 
                    ]))
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(my_ds,
    batch_size=batch_size, shuffle=True, **kwargs)


my_test_ds = MNISTAdderDataSet('/content/data/MNIST/processed', False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,)) 
                    ]))
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
test_loader = torch.utils.data.DataLoader(my_test_ds,
    batch_size=batch_size, shuffle=True, **kwargs)

In [8]:
model = MNISTAdderNet(start_channels=16,exponetate_layers=False).to(device)
#model(a.to(device))
img, input_adder_array, target, result_add = next(iter(train_loader))

In [9]:
pred_image, pred_sum = model(img.to(device), input_adder_array.to(device))
pred_image.shape, pred_sum.shape
F.nll_loss(pred_sum, result_add.to(device))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


(torch.Size([128, 10]), torch.Size([128, 19]))

## Train and Test function definitions.

In [11]:
### For 

from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    #pbar = tqdm(train_loader)
    for batch_idx, (img, input_adder_array, target, result_add) in enumerate(train_loader):
        #data, target = data.to(device), target.to(device)
        img = img.to(device)        
        input_adder_array = input_adder_array.to(device)
        target = target.to(device)
        result_add = result_add.to(device)

        optimizer.zero_grad()
        pred_image, pred_sum = model(img, input_adder_array)
        loss = F.nll_loss(pred_image, target) + F.nll_loss(pred_sum, result_add) ### We use two loss functions - one for MNIST class loss and one for predicting the result of addition.
        loss.backward()
        optimizer.step()



def test(model, device, test_loader):
    model.eval()
    class_loss = 0
    sum_loss = 0
    correct = 0
    correct_sum = 0
    with torch.no_grad():
        for img, input_adder_array, target, result_add in test_loader:
            # data, target = data.to(device), target.to(device)
            img = img.to(device)        
            input_adder_array = input_adder_array.to(device)
            target = target.to(device)
            result_add = result_add.to(device)
            # output = model(data)
            pred_image, pred_sum = model(img, input_adder_array)

            class_loss += F.nll_loss(pred_image, target, reduction='sum').item() 
            sum_loss += F.nll_loss(pred_sum, result_add, reduction='sum').item()
            # test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = pred_image.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            pred_sum_vals = pred_sum.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct_sum += pred_sum_vals.eq(result_add.view_as(pred_sum_vals)).sum().item()

    class_loss /= len(test_loader.dataset)
    sum_loss /= len(test_loader.dataset)

    print(f'Epoch: {epoch} Test set: Average class loss: {class_loss}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset)}%)')
    print(f'Epoch: {epoch} Test set: Average Sum Loss:{sum_loss} Accuracy: {correct_sum}/{len(test_loader.dataset)} ({100. * correct_sum / len(test_loader.dataset)}%)')
        # epoch, class_loss, sum_loss, correct, len(test_loader.dataset),
        # 100. * correct / len(test_loader.dataset)))

## Train/Test Loops
Accuracy on Images is ~99..64
Accuracy on predicted sum is ~98.72

In [12]:
model = MNISTAdderNet(start_channels=16,exponetate_layers=True).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) 


for epoch in range(1, 50):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 1 Test set: Average class loss: 0.029781136710895224, Accuracy: 9909/10000 (99.09%)
Epoch: 1 Test set: Average Sum Loss:2.1067971336364746 Accuracy: 2077/10000 (20.77%)
Epoch: 2 Test set: Average class loss: 0.02811167477183044, Accuracy: 9922/10000 (99.22%)
Epoch: 2 Test set: Average Sum Loss:1.6257107707977294 Accuracy: 3710/10000 (37.1%)
Epoch: 3 Test set: Average class loss: 0.021551052032783628, Accuracy: 9935/10000 (99.35%)
Epoch: 3 Test set: Average Sum Loss:1.3302183277130126 Accuracy: 4979/10000 (49.79%)
Epoch: 4 Test set: Average class loss: 0.020353321626968682, Accuracy: 9938/10000 (99.38%)
Epoch: 4 Test set: Average Sum Loss:1.1584116191864013 Accuracy: 5411/10000 (54.11%)
Epoch: 5 Test set: Average class loss: 0.023299573473073543, Accuracy: 9940/10000 (99.4%)
Epoch: 5 Test set: Average Sum Loss:1.0022976754188537 Accuracy: 6771/10000 (67.71%)
Epoch: 6 Test set: Average class loss: 0.02777718466236256, Accuracy: 9931/10000 (99.31%)
Epoch: 6 Test set: Average Sum Lo